In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


In [4]:

input_size = 64  
num_classes = 33  
batch_size = 32
num_epochs = 20
learning_rate = 0.001
validation_split = 0.2  

In [5]:


transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

dataset = datasets.ImageFolder(r"D:\Captcha\yolo+cnn\captcha_dataset_yolo", transform=transform)



In [4]:

dataset_size = len(dataset)
val_size = int(validation_split * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [6]:

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (input_size // 8) * (input_size // 8), 128), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [6]:

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            
            outputs = model(images)
            loss = criterion(outputs, labels)
            

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()

        train_loss /= len(train_loader.dataset)
        train_accuracy = train_correct / len(train_loader.dataset)


        model.eval()
        val_loss, val_correct = 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

torch.save(model.state_dict(), 'test_cnn.pth')


Epoch 1/20
Train Loss: 1.8607, Train Accuracy: 0.4848
Val Loss: 0.1535, Val Accuracy: 0.9583
Epoch 2/20
Train Loss: 0.2857, Train Accuracy: 0.9218
Val Loss: 0.0563, Val Accuracy: 0.9883
Epoch 3/20
Train Loss: 0.1556, Train Accuracy: 0.9559
Val Loss: 0.0323, Val Accuracy: 0.9950
Epoch 4/20
Train Loss: 0.1019, Train Accuracy: 0.9717
Val Loss: 0.0383, Val Accuracy: 0.9933
Epoch 5/20
Train Loss: 0.0994, Train Accuracy: 0.9705
Val Loss: 0.0305, Val Accuracy: 0.9967
Epoch 6/20
Train Loss: 0.0792, Train Accuracy: 0.9767
Val Loss: 0.0086, Val Accuracy: 0.9967
Epoch 7/20
Train Loss: 0.0692, Train Accuracy: 0.9817
Val Loss: 0.0062, Val Accuracy: 0.9983
Epoch 8/20
Train Loss: 0.0513, Train Accuracy: 0.9809
Val Loss: 0.0235, Val Accuracy: 0.9933
Epoch 9/20
Train Loss: 0.0370, Train Accuracy: 0.9892
Val Loss: 0.0319, Val Accuracy: 0.9950
Epoch 10/20
Train Loss: 0.0442, Train Accuracy: 0.9834
Val Loss: 0.0160, Val Accuracy: 0.9950
Epoch 11/20
Train Loss: 0.0449, Train Accuracy: 0.9867
Val Loss: 0.02